# LLM-Powered ISO 8583 Features

This notebook demonstrates the AI-powered features in iso8583sim:
- **MessageExplainer**: Explain messages in plain English
- **MessageGenerator**: Generate messages from natural language

## Prerequisites

Install an LLM provider:
```bash
pip install iso8583sim[anthropic]  # Claude
pip install iso8583sim[openai]     # GPT
pip install iso8583sim[google]     # Gemini
pip install iso8583sim[ollama]     # Local (Llama/Mistral)
pip install iso8583sim[llm]        # All providers
```

Set your API key:
```bash
export ANTHROPIC_API_KEY="sk-ant-..."
# or
export OPENAI_API_KEY="sk-..."
# or
export GOOGLE_API_KEY="..."
```

In [1]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.types import ISO8583Message
from iso8583sim.demo import generate_auth_request, generate_emv_auth, pretty_print

## Check Available Providers

First, let's see which LLM providers are available:

In [2]:
from iso8583sim.llm import list_available_providers, list_installed_providers

print("Installed providers (packages installed):")
print(f"  {list_installed_providers() or 'None'}")

print("\nAvailable providers (installed + configured):")
print(f"  {list_available_providers() or 'None - set API key env var'}")

Installed providers (packages installed):


  ['anthropic', 'openai', 'google', 'ollama']

Available providers (installed + configured):
  ['openai']


## MessageExplainer

The `MessageExplainer` takes an ISO 8583 message and provides a human-readable explanation.

In [3]:
from iso8583sim.llm import MessageExplainer

# Create an explainer (auto-detects available provider)
try:
    explainer = MessageExplainer()
    print(f"Using provider: {explainer.provider.name}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    explainer = None

Using provider: OpenAI


### Explain a Message Object

In [4]:
# Create a sample message
message = generate_auth_request(
    pan="4111111111111111",
    amount=10000,  # $100.00
    terminal_id="TERM0001",
    merchant_id="GASSTATION12345",
)

print("Message to explain:")
pretty_print(message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation:")
    print("=" * 60)
    explanation = explainer.explain(message)
    print(explanation)

Message to explain:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (9 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 000000
  F004 [NUMERIC] Amount, Transaction                 = 000000010000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2612
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = TERM0001
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = GASSTATION12345
  F049 [NUMERIC] Currency Code, Transaction          = 840

LLM Explanation:


Here's a breakdown and analysis of the provided ISO 8583 message:

1. **Transaction Type**:
   - **MTI 0100**: 
     - Digit 1: `0` - ISO 8583:1987 version.
     - Digit 2: `1` - Authorization message class, indicating it's seeking approval for a transaction.
     - Digit 3: `0` - Request function, meaning it's initiating a transaction.
     - Digit 4: `0` - Originating at the acquirer.
   - **Processing Code (F003: 000000)**: 
     - The first two digits `00` indicate a purchase transaction.

2. **Key Details**:
   - **Primary Account Number (F002: 411111******1111)**:
     - Indicates a Visa card, as it starts with `4`.
   - **Transaction Amount (F004: 000000010000)**:
     - Represents $100.00 USD since the amount is in minor units (cents) and the currency code (F049) is `840` (USD).
   - **Expiration Date (F014: 2612)**:
     - Indicates the card expires in December 2026.
   - **POS Entry Mode (F022: 051)**:
     - Indicates the transaction was made using a chip-enabled card.
   - 

### Explain a Raw Message String

In [5]:
builder = ISO8583Builder()
raw_message = builder.build(message)

print(f"Raw message: {raw_message[:80]}...")

if explainer:
    print("\nExplanation:")
    explanation = explainer.explain(raw_message)
    print(explanation)

Raw message: 01007024040000C080001641111111111111110000000000000100001234562612051TERM0001GAS...

Explanation:


Here's a detailed analysis and explanation of the ISO 8583 message:

1. **Transaction Type**:
   - **MTI (Message Type Indicator)**: 0100
     - *Digit 1: Version*: 0 - Version 1987
     - *Digit 2: Message Class*: 1 - Authorization
     - *Digit 3: Function*: 0 - Request
     - *Digit 4: Origin*: 0 - Acquirer
   - **Processing Code (F003)**: 000000 
     - Indicates a standard Purchase Transaction.

2. **Key Details**:
   - **Network**: VISA
     - PAN (F002): 411111******1111 indicates a VISA card, as it starts with 4.
   - **Transaction Amount (F004)**: 000000010000
     - This is an amount of 10,000 cents, equating to $100.00.
   - **Merchant Information**:
     - Terminal ID (F041): TERM0001
     - Merchant ID (F042): GASSTATION12345
   - **Card Expiry Date (F014)**: 2612
     - Expiration Date is December 2026.
   - **Transaction Time (F011)**: 123456
     - This is the System Trace Audit Number, used for identifying the transaction.
   - **POS Entry Mode (F022)**: 051
     - Ind

### Explain EMV/Chip Card Message

In [6]:
emv_message = generate_emv_auth(
    pan="4111111111111111",
    amount=25000,  # $250.00
    cryptogram="1234567890ABCDEF",
)

print("EMV Message:")
pretty_print(emv_message)

if explainer:
    print("\n" + "=" * 60)
    print("LLM Explanation of EMV Transaction:")
    print("=" * 60)
    explanation = explainer.explain(emv_message, verbose=True)
    print(explanation)

EMV Message:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (12 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 000000
  F004 [NUMERIC] Amount, Transaction                 = 000000025000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2612
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F023 [NUMERIC] Card Sequence Number                = 001
  F035 [LLVAR] Track 2 Data                        = 4111111111111111=26125010000000000000
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = TERM0001
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = MERCHANT123456 
  F049 [NUMERIC] Currency Code, Transaction          = 840
  F055 [LLLVAR] IC

Certainly! Here's a breakdown of the provided ISO 8583 message:

### 1. Transaction Type:

- **MTI (0100)**:
  - **Version**: 0 (1987 version of ISO 8583)
  - **Message Class**: 1 (Authorization message)
  - **Function**: 0 (Request)
  - **Origin**: 0 (Acquirer)
  - **Overall**: This is an authorization request from an acquirer to an issuer.

- **Processing Code (F003: 000000)**:
  - This indicates a standard purchase transaction (no special processing like cash or refund).

### 2. Key Details:

- **Card Information**:
  - **Primary Account Number (F002)**: 411111******1111
  - **Track 2 Data (F035)**: Indicates a VISA card (starts with 4). Expiration date and additional data encoded as part of the Track 2 data.
  - **Expiration Date (F014)**: 2612, which translates to December 2026.

- **Amount**:
  - **Transaction Amount (F004)**: 000000025000, which is $250.00 USD (since it is in minor units).

- **Merchant and POS**:
  - **Terminal ID (F041)**: TERM0001
  - **Merchant ID (F042)**: 

### Explain Specific Fields

In [7]:
if explainer:
    # Explain response codes
    print("Explaining Response Code 51:")
    print(explainer.explain_response_code("51"))

    print("\n" + "=" * 40)

    # Explain a specific field
    print("\nExplaining Field 22 (POS Entry Mode):")
    print(explainer.explain_field(22, "051"))

Explaining Response Code 51:


ISO 8583 response code "51" indicates "Insufficient Funds". This response is generated by the card issuer when the available balance in the cardholder's account is not sufficient to cover the amount of the transaction being attempted.

### 1. What this code means:

When a transaction is sent to the card issuer for approval, the issuer checks the cardholder's account to verify if there are enough funds to complete the transaction. A response code "51" signifies that the balance is not enough to authorize the purchase or withdrawal.

### 2. Common scenarios when this response is returned:

- The cardholder is attempting to make a purchase that exceeds their current available balance.
- The cardholder may have insufficient credit, in the case of a credit card, if they have already utilized most of their credit limit.
- Pending transactions that have not yet settled may reduce the Available Balance, triggering an insufficient funds status.
- There may be holds placed on the account (e.g., 

1. **What this field represents**:
   - Field 22 in the ISO 8583 message is known as the Point of Service (POS) Entry Mode. This field indicates how the cardholder's account data was entered into the payment system during the transaction. It gives information about the method and technology used to capture the card data.

2. **How to interpret the specific value**:
   - The value "051" can be broken down for detailed interpretation:
     - The first two digits, "05", represent the card entry mode, which indicates that the transaction was processed using a chip (integrated circuit card). This means the card was inserted into a chip-reading POS terminal.
     - The third digit, "1", typically indicates whether the cardholder’s signature was captured during the transaction. In many systems, "1" could mean signature capture was utilized.
   - Thus, "051" suggests the card was processed using a chip, and the transaction may have involved capturing the cardholder's signature.

3. **Any relev

## MessageGenerator

The `MessageGenerator` creates ISO 8583 messages from natural language descriptions.

In [8]:
from iso8583sim.llm import MessageGenerator

try:
    generator = MessageGenerator()
    print(f"Using provider: {generator.provider.name}")
except Exception as e:
    print(f"No LLM provider available: {e}")
    generator = None

Using provider: OpenAI


### Generate from Natural Language

In [9]:
if generator:
    # Generate a simple purchase
    description = "$50 VISA purchase at a coffee shop"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: $50 VISA purchase at a coffee shop

Generating message...



Generated Message:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (13 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 000000
  F004 [NUMERIC] Amount, Transaction                 = 000000005000
  F007 [NUMERIC] Transmission Date & Time (MMDDhhmms = 1024110000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F012 [NUMERIC] Time, Local Transaction (hhmmss)    = 110000
  F013 [NUMERIC] Date, Local Transaction (MMDD)      = 1024
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2512
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = COFFEE01
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = 123456789012345
  F043 [ALPHANUMERIC] Card Accep

In [10]:
if generator:
    # Generate a refund
    description = "Refund $25 to Mastercard ending in 4444 at ACME Store"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Refund Message:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: Refund $25 to Mastercard ending in 4444 at ACME Store

Generating message...



Generated Refund Message:
ISO 8583 Message - MTI: 0200

MTI Breakdown:
  Version:  0 (1987)
  Class:    2 (Financial)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (12 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 510510******5100
  F003 [NUMERIC] Processing Code                     = 200000
  F004 [NUMERIC] Amount, Transaction                 = 000000002500
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F012 [NUMERIC] Time, Local Transaction (hhmmss)    = 150000
  F013 [NUMERIC] Date, Local Transaction (MMDD)      = 1025
  F014 [NUMERIC] Date, Expiration (YYMM)             = 2505
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = ACME1234
  F042 [ALPHANUMERIC] Card Acceptor ID Code               = ACME00000000001
  F043 [ALPHANUMERIC] Card Acceptor Name/Location         = ACME Store                Anytown U

In [11]:
if generator:
    # Generate a balance inquiry
    description = "Balance inquiry for VISA card at ATM"
    print(f"Description: {description}")
    print("\nGenerating message...")

    try:
        message = generator.generate(description, validate=False)
        print("\nGenerated Balance Inquiry:")
        pretty_print(message)
    except Exception as e:
        print(f"Generation failed: {e}")

Description: Balance inquiry for VISA card at ATM

Generating message...



Generated Balance Inquiry:
ISO 8583 Message - MTI: 0100

MTI Breakdown:
  Version:  0 (1987)
  Class:    1 (Authorization)
  Function: 0 (Request)
  Origin:   0 (Acquirer)

Fields (9 data elements):
------------------------------------------------------------
  F002 [LLVAR] Primary Account Number (PAN)        = 411111******1111
  F003 [NUMERIC] Processing Code                     = 310000
  F004 [NUMERIC] Amount, Transaction                 = 000000000000
  F011 [NUMERIC] Systems Trace Audit Number (STAN)   = 123456
  F012 [NUMERIC] Time, Local Transaction (hhmmss)    = 123456
  F013 [NUMERIC] Date, Local Transaction (MMDD)      = 1123
  F022 [NUMERIC] Point of Service Entry Mode         = 051
  F041 [ALPHANUMERIC] Card Acceptor Terminal ID           = ATM12345
  F049 [NUMERIC] Currency Code, Transaction          = 840


### Suggest Missing Fields

In [12]:
if generator:
    # Create a partial message
    partial_message = ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: "4111111111111111",
            4: "000000010000",
        },
    )

    print("Partial Message:")
    print(f"  MTI: {partial_message.mti}")
    print(f"  Fields: {list(partial_message.fields.keys())}")

    print("\nSuggesting missing fields...")
    suggestions = generator.suggest_fields(partial_message)

    if suggestions:
        print("\nSuggested Fields:")
        for field_num, value in sorted(suggestions.items()):
            print(f"  F{field_num:03d}: {value}")
    else:
        print("No suggestions available")

Partial Message:
  MTI: 0100
  Fields: [0, 2, 4]

Suggesting missing fields...


No suggestions available


## Using Specific Providers

You can specify which LLM provider to use:

In [13]:
# Use a specific provider by name
try:
    from iso8583sim.llm import get_provider

    # Try different providers
    for provider_name in ["anthropic", "openai", "google", "ollama"]:
        try:
            provider = get_provider(provider_name)
            print(f"✓ {provider_name}: {provider.model}")
        except Exception as e:
            print(f"✗ {provider_name}: {e}")
except Exception as e:
    print(f"Error: {e}")

✗ anthropic: Anthropic provider requires 'anthropic' package. Install with: pip install iso8583sim[anthropic]
✓ openai: gpt-4o
✗ google: Google provider requires 'google-generativeai' package. Install with: pip install iso8583sim[google]
✗ ollama: Ollama provider requires 'ollama' package. Install with: pip install iso8583sim[ollama]


In [14]:
# Use provider with custom settings
try:
    from iso8583sim.llm.providers.anthropic import AnthropicProvider

    # Use a specific model
    provider = AnthropicProvider(model="claude-3-haiku-20240307")
    explainer = MessageExplainer(provider=provider)

    print(f"Using: {explainer.provider.name} / {explainer.provider.model}")
except Exception as e:
    print(f"Could not create custom provider: {e}")

Could not create custom provider: Anthropic provider requires 'anthropic' package. Install with: pip install iso8583sim[anthropic]


## Error Handling

The LLM module provides clear error messages:

In [15]:
from iso8583sim.llm import LLMError, ProviderConfigError, ProviderNotAvailableError

# Example error handling
try:
    from iso8583sim.llm import get_provider

    provider = get_provider("invalid_provider")
except ProviderConfigError as e:
    print(f"Config Error: {e}")
except ProviderNotAvailableError as e:
    print(f"Not Available: {e}")
except LLMError as e:
    print(f"LLM Error: {e}")

Config Error: Unknown provider: invalid_provider. Available: anthropic, openai, google, ollama


## Next Steps

- Try different LLM providers to see how they compare
- Use the generator to create test message datasets
- Combine with validation to ensure generated messages are correct
- Check out the API reference for more advanced usage